[![Binder](https://mybinder.org/badge_logo.svg)](https://nbviewer.org/github/Sistemas-Multimedia/Sistemas-Multimedia.github.io/blob/master/milestones/10-ME/full_search_dense_ME.ipynb)

# Full search dense motion estimation

In [ ]:
import numpy as np
import image_3 as frame
import YCoCg as YUV
import motion
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['text.usetex'] = True
!ln -sf ~/MRVC/src/LP.py .
import LP
import cv2 as cv
from common import show_frame
from common import show_vectors
from common import normalize
!ln -sf ~/quantization/information.py .
import information

If we extrapolate the idea used in `full_search_block_based_BE()` when the `block_side==1`, there are a huge amount of redundant substractions when adjacent blocks are processed. In the following implementation, the subtractions are performed only once because all the "blocks" (that in this case are of one pixel) are subtracted at each seaching point (one for search area). Another characteristic of this algorithm is that the error frames are smoothed to increase the correlation between the motion vectors.

In [ ]:
# Copiado de MRVC/src/motion.py
OVERLAPPING_AREA_SIDE = 17
SEARCH_RANGE = 32
def full_search_dense_ME(P, R, sr=SEARCH_RANGE, oas=OVERLAPPING_AREA_SIDE):
    assert OVERLAPPING_AREA_SIDE % 2 != 0 # This a requirement of cv.GaussianBLur
    extended_R = np.zeros((R.shape[0] + sr, R.shape[1] + sr)) # Ojo, probar extension
    extended_R[sr//2 : R.shape[0] + sr//2, sr//2 : R.shape[1] + sr//2] = R
    MVs = np.zeros((P.shape[0], P.shape[1], 2), dtype=np.int8)
    min_error = np.full((P.shape[0], P.shape[1]), 255, dtype=np.uint8)
    for y in range(sr):
        print(f"{y}/{sr - 1}", end='\r')
        for x in range(sr):
            error = extended_R[y : P.shape[0] + y, x : P.shape[1] + x] - P
            a_error = abs(error) # Ojo probar MSE
            blur_a_error = cv.GaussianBlur(a_error, (oas, oas), 0).astype(np.int)
            which_min = blur_a_error <= min_error
            MVs[:,:,0] = np.where(which_min, x - sr//2, MVs[:,:,0])
            MVs[:,:,1] = np.where(which_min, y - sr//2, MVs[:,:,1])
            min_error = np.minimum(min_error, blur_a_error)
    return MVs.astype(np.float)

## Testing with moving circles (max_abs_motion=1)

In [ ]:
R = frame.read("/home/vruiz/MRVC/sequences/moving_circles/", 0)
R_Y = YUV.from_RGB(R.astype(np.int16))[...,0]
P = frame.read("/home/vruiz/MRVC/sequences/moving_circles/", 1)
P_Y = YUV.from_RGB(P.astype(np.int16))[...,0]

In [ ]:
MVs = full_search_dense_ME(P_Y, R_Y, 16, 17)

In [ ]:
show_vectors(MVs)

In [ ]:
hatP = motion.make_prediction(R, MVs.astype(np.float32))

In [ ]:
show_frame(R, "reference ${\mathbf R}$")
show_frame(P, "predicted ${\mathbf P}$")
show_frame(hatP, "prediction $\hat{\mathbf P}$")
show_frame(normalize(hatP - P), "prediction error")

## Testing with moving circles (max_abs_motion=2)

In [ ]:
R = frame.read("/home/vruiz/MRVC/sequences/moving_circles/", 0)
R_Y = YUV.from_RGB(R.astype(np.int16))[...,0]
P = frame.read("/home/vruiz/MRVC/sequences/moving_circles/", 2)
P_Y = YUV.from_RGB(P.astype(np.int16))[...,0]

In [ ]:
MVs = full_search_dense_ME(P_Y, R_Y, 16, 17)

In [ ]:
show_vectors(MVs)

In [ ]:
hatP = motion.make_prediction(R, MVs.astype(np.float32))

In [ ]:
show_frame(R, "reference ${\mathbf R}$")
show_frame(P, "predicted ${\mathbf P}$")
show_frame(hatP, "prediction $\hat{\mathbf P}$")
show_frame(normalize(hatP - P), "prediction error")

## Testing with mobile

In [ ]:
R = frame.read("/home/vruiz/MRVC/sequences/mobile/", 0)
R_Y = YUV.from_RGB(R.astype(np.int16))[...,0]
P = frame.read("/home/vruiz/MRVC/sequences/mobile/", 1)
P_Y = YUV.from_RGB(P.astype(np.int16))[...,0]

In [ ]:
MVs = full_search_dense_ME(P_Y, R_Y, 16, 17)

In [ ]:
show_vectors(MVs[::10, ::10])

In [ ]:
hatP = motion.make_prediction(R, MVs.astype(np.float32))

In [ ]:
show_frame(R, "reference ${\mathbf R}$")
show_frame(P, "predicted ${\mathbf P}$")
show_frame(hatP, "prediction $\hat{\mathbf P}$")
show_frame(normalize(hatP - P), "prediction error")

## Testing with bus

In [ ]:
R = frame.read("/home/vruiz/MRVC/sequences/bus/", 0)
R_Y = YUV.from_RGB(R.astype(np.int16))[...,0]
P = frame.read("/home/vruiz/MRVC/sequences/bus/", 1)
P_Y = YUV.from_RGB(P.astype(np.int16))[...,0]

In [ ]:
MVs = full_search_dense_ME(P_Y, R_Y, 16, 17)

In [ ]:
show_vectors(MVs[::10, ::10])

In [ ]:
hatP = motion.make_prediction(R, MVs.astype(np.float32))

In [ ]:
show_frame(R, "reference ${\mathbf R}$")
show_frame(P, "predicted ${\mathbf P}$")
show_frame(hatP, "prediction $\hat{\mathbf P}$")
show_frame(normalize(hatP - P), "prediction error")

## Testing with a tile of stockholm

In [ ]:
prefix = "/home/vruiz/MRVC/sequences/stockholm/"
R = frame.read(prefix, 0)[100:356,100:612]
P = frame.read(prefix, 1)[100:356,100:612]
R_Y = YUV.from_RGB(R.astype(np.int16))[...,0]
P_Y = YUV.from_RGB(P.astype(np.int16))[...,0]

In [ ]:
MVs = full_search_dense_ME(P_Y, R_Y, 16, 17)

In [ ]:
entropy = information.entropy(MVs.flatten())
show_vectors(MVs[::10, ::10], title="${\mathbf V}$ (dense ME) " + f"entropy={entropy:1.2f} bits/component" + f", {MVs.size} components")

In [ ]:
hat_P = motion.make_prediction(R, MVs.astype(np.float32))

In [ ]:
show_frame(R, "${\mathbf R}$")
show_frame(P, "${\mathbf P}$")
show_frame(hat_P, "$\hat{\mathbf P}$ (dense ME)")
P_hat_P = P - hat_P + 128
entropy = information.entropy(P_hat_P.flatten())
show_frame(P_hat_P, "${\mathbf P}$ - $\hat{\mathbf P}$ (dense ME)" + f" entropy={entropy:1.2f} bits/pixel")